In [1]:
import numpy as np
import pandas as pd

In [2]:
movies=pd.read_csv("tmdb_5000_movies.csv")
credits=pd.read_csv("tmdb_5000_credits.csv")

In [3]:
movies.head()

credits.head()

movies.shape

credits.shape

(4803, 4)

In [4]:
movies.rename(columns={'id':"movie_id"},inplace=True)

data=movies.merge(credits,on='movie_id')

In [5]:
data.columns

data.rename(columns={'title_x':'title'},inplace=True)

data.columns

data.sample(1)

,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,title_y,cast,crew
4289,0,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 35, ""n...",NaN,278316,[],en,Da Sweet Blood of Jesus,A movie about human beings who are addicted to...,4.480579,"[{""name"": ""40 Acres & A Mule Filmworks"", ""id"":...",...,123.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Da Sweet Blood of Jesus,4.1,13,Da Sweet Blood of Jesus,"[{""cast_id"": 14, ""character"": """", ""credit_id"":...","[{""credit_id"": ""53a821770e0a26144500598e"", ""de..."


In [6]:
columns_to_remove=['budget','homepage','original_language','original_title','popularity','production_countries','release_date','revenue','runtime','spoken_languages','status','vote_average','vote_count','title_y']

data=data.drop(columns=columns_to_remove,axis=1)

data.head()

from ast import literal_eval

data['genres'].values

def extract_names(data):
    names=[]
    for items in literal_eval(data):
        names.append(items['name'])
    return names

data['genres']=data['genres'].apply(extract_names)

data.head(1)

data['keywords']=data['keywords'].apply(extract_names)

data.head()

data['production_companies'][0]

data['production_companies']=data['production_companies'].apply(extract_names)

def extract_top_names(data):
    names=[]
    counter=0
    for items in literal_eval(data):
        if counter<3:
            names.append(items['name'])
            counter+=1
    return names

data['cast']=data['cast'].apply(extract_top_names)

data.sample(2)

def fetch_director(data):
    name=[]
    for item in literal_eval(data):
        if item['job']=='Director':
            name.append(item['name'])
    return name

data['crew']=data['crew'].apply(fetch_director)

data.sample()

data['overview']

import string

data['overview']=data['overview'].str.translate(str.maketrans('', '',string.punctuation))

data.sample(1)

data['tagline']=data['tagline'].str.lower().str.translate(str.maketrans('', '',string.punctuation))

data['tagline'].isnull().sum()

data.drop(columns='tagline',axis=1,inplace=True)

data.sample()

from nltk.tokenize import word_tokenize

def tokenizer(data):
    for item in data:
        return word_tokenize(item)

data=data[~data['overview'].isnull()]

data['overview']=data['overview'].apply(word_tokenize)

def remove_spaces(data):
    names=[]
    for item in data:
        names.append(item.replace(" ",""))
    return names

data.sample()

data['cast']=data['cast'].apply(remove_spaces)

data['cast']

data['crew']=data['crew'].apply(remove_spaces)

data['production_companies']=data['production_companies'].apply(remove_spaces)

data.head(5)

data.head()

data.sample()

data['tags']=data['genres']+data['keywords']+data['overview']+data['production_companies']+data['cast']+data['crew']

data.drop(columns=['genres','keywords','overview','production_companies','cast','crew'],axis=1,inplace=True)

data.sample(5)

data['tags']=data['tags'].apply(remove_spaces)

In [7]:
def to_lower(data):
    lst=[]
    for item in data:
        lst.append(item.lower())
    return lst

In [8]:
data['tags']=data['tags'].apply(to_lower)

data.sample(5)

data['tags']=data['tags'].apply(lambda x:" ".join(x))

data.head()

,movie_id,title,tags
0,19995,Avatar,action adventure fantasy sciencefiction cultur...
1,285,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drugabuse exoti...
2,206647,Spectre,action adventure crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,action crime drama thriller dccomics crimefigh...
4,49529,John Carter,action adventure sciencefiction basedonnovel m...


In [9]:
data['tags'][0]

'action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d in the 22nd century a paraplegic marine is dispatched to the moon pandora on a unique mission but becomes torn between following orders and protecting an alien civilization ingeniousfilmpartners twentiethcenturyfoxfilmcorporation duneentertainment lightstormentertainment samworthington zoesaldana sigourneyweaver jamescameron'

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
vectorizer=CountVectorizer(max_features=5000,stop_words='english')

In [12]:
vectors=vectorizer.fit_transform(data['tags']).toarray()

In [13]:
import nltk

In [14]:
from nltk.stem.porter import PorterStemmer

In [15]:
stemmer=PorterStemmer()

In [16]:
def stem_text(data):
    l=[]
    for element in data.split():
        l.append(stemmer.stem(element))
    return " ".join(l)

In [17]:
data['tags']=data['tags'].apply(stem_text)

In [18]:
vectors=vectorizer.fit_transform(data['tags']).toarray()

In [19]:
pd.set_option('display.max_columns',None)

In [20]:
vectorizer.get_feature_names_out()

array(['007', '10', '100', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [21]:
data['tags'][0]

'action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d in the 22nd centuri a parapleg marin is dispatch to the moon pandora on a uniqu mission but becom torn between follow order and protect an alien civil ingeniousfilmpartn twentiethcenturyfoxfilmcorpor duneentertain lightstormentertain samworthington zoesaldana sigourneyweav jamescameron'

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
vectors.shape

(4800, 5000)

In [24]:
similarity=cosine_similarity(vectors)

In [25]:
data.sample(1)

,movie_id,title,tags
4088,80,Before Sunset,drama romanc pari journalist dialogu talk soul...


In [30]:
data[data['title']=='Avatar'].index[0]

np.int64(0)

In [35]:
def recommend(movie):
    movie_index=data[data['title']==movie].index[0]
    distances=similarity[movie_index]
    movie_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for movie in movie_list:
        print(data.iloc[movie[0]].title)

In [36]:
recommend('Avatar')

Aliens
Titan A.E.
Aliens vs Predator: Requiem
Meet Dave
Independence Day


In [37]:
import pickle

In [40]:
pickle.dump(data,open('movies.pkl','wb'))

In [42]:
pickle.dump(similarity,open('similarity.pkl','wb'))